Current example follows this [document](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html) .

In [1]:
# 1. Prepare dependencies
import mnist

import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn


In [2]:
# 2. prepare training and testing dataloader

mnist.datasets_url="http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/"

train_images = mnist.train_images()
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()


train_images_tensor = torch.Tensor(train_images)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.int64)
test_images_tensor = torch.Tensor(test_images)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.int64)

train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_images_tensor, test_labels_tensor)


train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()

        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU(),
            nn.Linear(10, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=10, bias=True)
    (3): ReLU()
    (4): Linear(in_features=10, out_features=10, bias=True)
  )
)

In [4]:
# Define hyperparameters

learning_rate = 1e-3
batch_size = 64
epochs = 5

In [5]:
# Define the train_loop function

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        
        
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f'loss: {loss:>7f} [{current:>5d}/{size:>5d}]')


def test_loop(dataloader, model, loss_fn):
    
    model.eval()

    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y ).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /=size

    print(f'Test Error: \n Accurary: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n')






In [6]:
# 3. training model

loss_fn = nn.CrossEntropyLoss()

optimizer =torch.optim.SGD(model.parameters(), lr=learning_rate)


epochs = 10

for t in range(epochs):
    print(f'Epoch {t+1}\n--------------------')
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)




Epoch 1
--------------------
loss: 16.390299 [   64/60000]
loss: 2.319577 [ 6464/60000]
loss: 2.349446 [12864/60000]
loss: 2.290609 [19264/60000]
loss: 2.340511 [25664/60000]
loss: 2.343422 [32064/60000]
loss: 2.314474 [38464/60000]
loss: 2.355653 [44864/60000]
loss: 2.300287 [51264/60000]
loss: 2.276902 [57664/60000]
Test Error: 
 Accurary: 10.0%, Avg loss: 2.320933 

Epoch 2
--------------------
loss: 2.340177 [   64/60000]
loss: 2.285198 [ 6464/60000]
loss: 2.311335 [12864/60000]
loss: 2.320681 [19264/60000]
loss: 2.317040 [25664/60000]
loss: 2.326154 [32064/60000]
loss: 2.318019 [38464/60000]
loss: 2.329008 [44864/60000]
loss: 2.210922 [51264/60000]
loss: 2.333088 [57664/60000]
Test Error: 
 Accurary: 17.8%, Avg loss: 2.185964 

Epoch 3
--------------------
loss: 2.474085 [   64/60000]
loss: 2.101439 [ 6464/60000]
loss: 2.166438 [12864/60000]
loss: 2.236853 [19264/60000]
loss: 2.242551 [25664/60000]
loss: 1.951383 [32064/60000]
loss: 2.203146 [38464/60000]
loss: 2.139006 [44864/600

In [7]:

# model.state_dict()

torch.save(model.state_dict(), './model/main.pth')


OrderedDict([('linear_relu_stack.0.weight',
              tensor([[-1.6768e-02, -5.6354e-03, -7.6049e-03,  ...,  1.8614e-02,
                        1.4288e-02,  4.4046e-05],
                      [ 7.1397e-03, -2.5362e-02, -2.0448e-02,  ...,  2.6557e-02,
                       -3.1808e-02,  3.3969e-02],
                      [ 1.0782e-02,  5.2428e-03, -8.2378e-03,  ..., -8.2695e-03,
                        3.3830e-02, -2.8091e-03],
                      ...,
                      [ 1.6517e-03, -1.6262e-02, -3.2337e-02,  ...,  7.5016e-03,
                       -3.2627e-02,  2.1107e-02],
                      [ 1.7445e-02, -5.3874e-03, -3.1857e-02,  ..., -4.3365e-03,
                       -2.5193e-02, -3.0141e-02],
                      [ 1.7908e-02,  4.9070e-03,  1.4841e-02,  ...,  2.4168e-02,
                       -2.2036e-02, -1.0493e-03]])),
             ('linear_relu_stack.0.bias',
              tensor([ 4.8575e-03, -3.2404e-04, -3.0490e-02,  1.4438e-02,  2.2419e-02,
           